# MVVM + Hexagonal in SEVA — Part 2 (View ↔ ViewModel ↔ UseCase)

## What we build in this notebook

We build one small Tkinter app that can:
- Start a demo experiment run
- Poll progress
- Show status text + progress bar

We keep the **polling example** exactly as before, but restructure it using **dependency injection**:
- View gets its dependencies (variables + callbacks) in `__init__`.
- ViewModel owns state + commands.
- UseCase runs the workflow and depends on `JobPort`.

As in **Part 1**, adapters were introduced there. Here we reuse **`JobRestMock`** to keep focus on MVVM.

In [ ]:
from pathlib import Path
import sys

# Allow notebook execution from docs/ while importing the project package.
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

## 1) View + Dependency Injection (DI)

The View is constructed with **dependencies injected**:
- `status_text` (StringVar)
- `progress_pct` (DoubleVar)
- `on_start`, `on_poll` callbacks

This mirrors the glueing style used in `main.py`: create dependencies first, then construct the View.

In [ ]:
import tkinter as tk
from tkinter import ttk
from dataclasses import dataclass


@dataclass
class StartVmState:
    status_text: str = "Idle"
    progress_pct: float = 0.0
    is_running: bool = False
    start_clicks: int = 0


class StartViewClassic:
    """Classic View: dependencies injected in __init__ (no business logic)."""

    def __init__(self, root: tk.Tk, status_text: tk.StringVar, progress_pct: tk.DoubleVar, on_start, on_poll):
        self.root = root
        self.root.title("MVVM Demo — Classic View")

        outer = ttk.Frame(root, padding=12)
        outer.pack(fill="both", expand=True)

        ttk.Label(outer, text="Experiment status").pack(anchor="w")
        ttk.Label(outer, textvariable=status_text).pack(anchor="w", pady=(2, 6))
        ttk.Progressbar(outer, maximum=100, variable=progress_pct, length=300).pack(anchor="w", pady=(0, 8))

        row = ttk.Frame(outer)
        row.pack(anchor="w")
        ttk.Button(row, text="Start", command=on_start).pack(side="left", padx=(0, 6))
        ttk.Button(row, text="Poll", command=on_poll).pack(side="left")

## 2) ViewModel (state + commands)

ViewModel holds the UI state and exposes command methods for the View.
This is still a pure UI layer: no network/filesystem I/O.

In [ ]:
class StartViewModelStepB:
    """ViewModel-only step (no use case yet)."""

    def __init__(self):
        self.state = StartVmState()
        # These variables are injected into the View.
        self.status_text = tk.StringVar(value=self.state.status_text)
        self.progress_pct = tk.DoubleVar(value=self.state.progress_pct)

    def can_start(self) -> bool:
        return not self.state.is_running

    def _sync(self):
        # Keep tkinter variables in sync with ViewModel state.
        self.status_text.set(self.state.status_text)
        self.progress_pct.set(self.state.progress_pct)

    def on_start_clicked(self):
        if not self.can_start():
            self.state.status_text = "Already running"
            self._sync()
            return

        self.state.is_running = True
        self.state.start_clicks += 1
        self.state.progress_pct = 0.0
        self.state.status_text = f"Run #{self.state.start_clicks} started"
        self._sync()

    def on_poll_clicked(self):
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            self._sync()
            return

        self.state.progress_pct = min(100.0, self.state.progress_pct + 10.0)
        self.state.status_text = f"Running... {self.state.progress_pct:.0f}%"

        if self.state.progress_pct >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"

        self._sync()

In [ ]:
# Step 2 assembled app: View + ViewModel + DI
root = tk.Tk()
vm_b = StartViewModelStepB()
view = StartViewClassic(root, vm_b.status_text, vm_b.progress_pct, vm_b.on_start_clicked, vm_b.on_poll_clicked)
root.mainloop()

## 3) UseCase (port-based) + ViewModel orchestration

The polling example stays the same, but UseCase now depends on `JobPort` (interface).
We inject `JobRestMock()` to keep this tutorial offline.

In [ ]:
from seva.adapters.job_rest_mock import JobRestMock
from seva.domain.ports import JobPort


class StartAndPollProgressUseCase:
    """UseCase: start a group and poll progress through JobPort."""

    def __init__(self, job_port: JobPort):
        self.job_port = job_port
        self.active_group_id = None

    def start(self) -> str:
        plan = JobRestMock.example_plan()
        group_id, _runs = self.job_port.start_batch(plan)
        self.active_group_id = group_id
        return group_id

    def poll_progress_pct(self) -> float:
        if not self.active_group_id:
            return 0.0

        snapshot = self.job_port.poll_group(self.active_group_id)

        # Simplified extraction for tutorial readability.
        for box in snapshot.get("boxes", {}).values():
            runs = box.get("runs", [])
            if runs:
                return float(runs[0].get("progress_pct", 0.0))
        return 0.0

In [ ]:
class StartViewModel:
    """ViewModel: orchestrates use-case calls + maps results to UI state."""

    def __init__(self, use_case: StartAndPollProgressUseCase):
        self.use_case = use_case
        self.state = StartVmState()
        self.status_text = tk.StringVar(value=self.state.status_text)
        self.progress_pct = tk.DoubleVar(value=self.state.progress_pct)

    def can_start(self) -> bool:
        return not self.state.is_running

    def _sync(self):
        self.status_text.set(self.state.status_text)
        self.progress_pct.set(self.state.progress_pct)

    def on_start_clicked(self):
        if not self.can_start():
            self.state.status_text = "Already running"
            self._sync()
            return

        group_id = self.use_case.start()
        self.state.start_clicks += 1
        self.state.is_running = True
        self.state.progress_pct = 0.0
        self.state.status_text = f"Started group: {group_id}"
        self._sync()

    def on_poll_clicked(self):
        if not self.state.is_running:
            self.state.status_text = "Idle (start first)"
            self._sync()
            return

        progress = self.use_case.poll_progress_pct()
        self.state.progress_pct = progress

        if progress >= 100.0:
            self.state.is_running = False
            self.state.status_text = "Done"
        else:
            self.state.status_text = f"Running... {progress:.1f}%"

        self._sync()

### Why `root.after` is not inside ViewModel

Scheduling is kept in the runner/composition function.
That keeps View and ViewModel simpler and easier to test.

In [ ]:
# Step 3 assembled app: View + ViewModel + UseCase + scheduler
root = tk.Tk()
uc = StartAndPollProgressUseCase(JobRestMock())
vm = StartViewModel(uc)
view = StartViewClassic(root, vm.status_text, vm.progress_pct, vm.on_start_clicked, vm.on_poll_clicked)


def _tick_poll():
    if vm.state.is_running:
        vm.on_poll_clicked()
    root.after(1000, _tick_poll)


_tick_poll()
root.mainloop()

## 4) Swap to a visually distinct (more modern) View

Same VM, same UseCase, same DI pattern — only View class changes.

In [ ]:
class StartViewModern:
    """More visually distinct view using themed/card-like layout."""

    def __init__(self, root: tk.Tk, status_text: tk.StringVar, progress_pct: tk.DoubleVar, on_start, on_poll):
        self.root = root
        self.root.title("MVVM Demo — Modern View")

        style = ttk.Style(root)
        if "clam" in style.theme_names():
            style.theme_use("clam")

        style.configure("Card.TFrame", padding=14)
        style.configure("Title.TLabel", font=("TkDefaultFont", 12, "bold"))

        shell = ttk.Frame(root, padding=16)
        shell.pack(fill="both", expand=True)

        card = ttk.Frame(shell, style="Card.TFrame")
        card.pack(fill="x")

        ttk.Label(card, text="Experiment Runner", style="Title.TLabel").pack(anchor="w")
        ttk.Label(card, text="Status").pack(anchor="w", pady=(8, 0))
        ttk.Label(card, textvariable=status_text).pack(anchor="w", pady=(0, 8))
        ttk.Progressbar(card, maximum=100, variable=progress_pct, length=320).pack(anchor="w", pady=(0, 10))

        action_row = ttk.Frame(card)
        action_row.pack(anchor="w")
        ttk.Button(action_row, text="▶ Start", command=on_start).pack(side="left", padx=(0, 8))
        ttk.Button(action_row, text="⟳ Poll", command=on_poll).pack(side="left")

In [ ]:
# Step 4 assembled app: same VM/UseCase, modern View (DI in constructor)
root = tk.Tk()
uc_m = StartAndPollProgressUseCase(JobRestMock())
vm_m = StartViewModel(uc_m)
modern_view = StartViewModern(root, vm_m.status_text, vm_m.progress_pct, vm_m.on_start_clicked, vm_m.on_poll_clicked)


def _tick_poll_modern():
    if vm_m.state.is_running:
        vm_m.on_poll_clicked()
    root.after(1000, _tick_poll_modern)


_tick_poll_modern()
root.mainloop()